# Final Project Submission

* **Student name**: Sara Robinson
* **Student pace**: *self paced*/part time/full time
* **Scheduled project review date/time**: 10/29/2021 18:00
* **Instructor name**: Jeff Herman
* **Project Reviewer**: James Irving
* **Notebook**: 2/4

# Introduction

The purpose of this project is to do a thorough exploratory data analysis of the characters and scripts of the TV show How I Met Your Mother, as well as create a model that can predict who said which line. The data is from the website https://transcripts.foreverdreaming.org/viewforum.php?f=177 where there are nine pages containing the 208 episodes from nine seasons that the show ran.

This is the second notebook for my Capstone Project. This notebook includes preprocessing of the text data to prepare it for exploring/modeling, as well as preparing the data for a SQL database so it is easier to explore in the next notebook. This is a short notebook and the rest of the EDA is not included here because some of the code takes a long time to run and I want to keep all of the EDA together and organized.

## Import Libraries

In the following cell I import the necessary libraries for this notebook.

In [1]:
import pandas as pd
import re
from textblob import TextBlob
from tqdm.notebook import tqdm

In [2]:
df = pd.read_csv('HIMYM.csv') #Reading in dataset
df.head() #Checking first five rows

,Season,Episode,Character,Line
0,1,1,Marshall,(Opens ring) Will you marry me.
1,1,1,Ted,"Yes, perfect! And then you're engaged, you pop..."
2,1,1,Marshall,"Got it. Thanks for helping me plan this out, Ted."
3,1,1,Ted,"Dude, are you kidding? It's you and Lily! I've..."
4,1,1,Marshall,"(laughs) yeah, sorry. We thought you were asleep."


Let's take a look at the df info and see if there are any duplicates.

In [3]:
print(df.info()) #Seeing information about our data

print(df.duplicated().sum()) #Checking to see if any rows are duplicated

print(df.isna().sum()) #Checking for null entries

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4739072 entries, 0 to 4739071
Data columns (total 4 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   Season     int64 
 1   Episode    int64 
 2   Character  object
 3   Line       object
dtypes: int64(2), object(2)
memory usage: 144.6+ MB
None
4716597
Season       0
Episode      0
Character    0
Line         0
dtype: int64


Golly gee willikers! No wonder everything took so long to run originally! It most likely was because it ran through a counter up to 208, so it went through every single episode 208 times. At this point I'm not going to go back and change it since I was scraping the website over and over, but what I can do is fix my data frame so the rest of this project can run smoothly. I'm going to drop the duplicates and then continue on with this notebook.

In [4]:
df.drop_duplicates(inplace = True) #Removing duplicates

print(df.info()) #Seeing df info

print(df.duplicated().sum()) #Checking duplicate count

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22475 entries, 0 to 22783
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Season     22475 non-null  int64 
 1   Episode    22475 non-null  int64 
 2   Character  22475 non-null  object
 3   Line       22475 non-null  object
dtypes: int64(2), object(2)
memory usage: 877.9+ KB
None
0


In [5]:
df.head() #Looking at head of df

,Season,Episode,Character,Line
0,1,1,Marshall,(Opens ring) Will you marry me.
1,1,1,Ted,"Yes, perfect! And then you're engaged, you pop..."
2,1,1,Marshall,"Got it. Thanks for helping me plan this out, Ted."
3,1,1,Ted,"Dude, are you kidding? It's you and Lily! I've..."
4,1,1,Marshall,"(laughs) yeah, sorry. We thought you were asleep."


That's much better and makes much more sense; 4 1/2 million lines for a TV series wasn't necessarily adding up, especially because I only was counting the lines of the 5 main characters. Now let's move on to preprocessing.

# Text Preprocessing

Before we can do the fun part of exploring this plethora of data, we need to clean up the Line column. Based on how the data was read into the DF in the previous notebook, there shouldn't be any issues other than removing the "motions" from the lines, aka everything in parentheses. Part of me wants to add a column and just push these motions into a new column so we can look at motions like high-fives (as we know, Barney loves to high-five), but this will certainly result in a majority of NaN entries as not every single line includes a motion. I guess it couldn't hurt to make the column for it anyway in case I want to use it, and if I don't then no harm and if I do then it's available and ready! It's been decided, thanks for following.

In [6]:
df[df['Line'].str.contains('\([^)]*\)')] #Checking regex pattern by looking at lines in df that contain it

,Season,Episode,Character,Line
0,1,1,Marshall,(Opens ring) Will you marry me.
4,1,1,Marshall,"(laughs) yeah, sorry. We thought you were asleep."
7,1,1,Barney,"(on the phone) hey, so you know how I've alway..."
16,1,1,Barney,"Ted, I'm going to teach you how to live. (Ted'..."
20,1,1,Barney,"Lesson two, get a suit. Suits are cool. (Point..."
...,...,...,...,...
22475,7,2,Marshall,"If I did not work, I could finish... in kind, ..."
22485,7,2,Ted,I just had two rencards following. With two ad...
22609,7,3,Ted,"Oh, no. How have I landed on it? Salvation. Te..."
22612,7,3,Marshall,"If I did not work, I could finish... in kind, ..."


In [7]:
df['Motion'] = df['Line'].apply(lambda x: re.findall(r'\([^)]*\)', x)) #Making column of motions

df.Line = df.Line.apply(lambda x: re.sub(r'\([^)]*\)', '', x)) #Removing all motions from Line column

df[df['Line'].str.contains('\([^)]*\)')] #Checking to make sure all motions were removed from Line column

,Season,Episode,Character,Line,Motion


In [8]:
df.head() #Checking df to make sure all columns were created properly

,Season,Episode,Character,Line,Motion
0,1,1,Marshall,Will you marry me.,[(Opens ring)]
1,1,1,Ted,"Yes, perfect! And then you're engaged, you pop...",[]
2,1,1,Marshall,"Got it. Thanks for helping me plan this out, Ted.",[]
3,1,1,Ted,"Dude, are you kidding? It's you and Lily! I've...",[]
4,1,1,Marshall,"yeah, sorry. We thought you were asleep.",[(laughs)]


## Sentiment

We can use TextBlob to calculate a sentiment score for each line. In the following cell we'll create a column that will contain this score for each line. Later on we can use these scores in our EDA, but for now we'll just calculate it. The following cell will be running through our DF that has over 4 1/2 million rows, so it may take awhile to run.

In [9]:
tqdm.pandas() #Setting tqdm pandas so we can see a progress bar as it loads

df['Polarity'] = df['Line'].progress_apply(lambda x: TextBlob(x).sentiment[0]) #Creating column for polarity score

df['Subjectivity'] = df['Line'].progress_apply(lambda x: TextBlob(x).sentiment[1]) #Creating column for subject score

/opt/anaconda3/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [10]:
df.head() #Checking df to make sure column was added properly

,Season,Episode,Character,Line,Motion,Polarity,Subjectivity
0,1,1,Marshall,Will you marry me.,[(Opens ring)],0.00000,0.000000
1,1,1,Ted,"Yes, perfect! And then you're engaged, you pop...",[],1.00000,1.000000
2,1,1,Marshall,"Got it. Thanks for helping me plan this out, Ted.",[],0.20000,0.200000
3,1,1,Ted,"Dude, are you kidding? It's you and Lily! I've...",[],0.09375,0.285417
4,1,1,Marshall,"yeah, sorry. We thought you were asleep.",[(laughs)],-0.50000,1.000000


I'm going to save this new DF to a csv file, then I'm going to create a database file from it on my desktop using DB Browser for SQLite so I can start exploring the data using SQL, which will be easier to navigate and smoother than working directly on the DF using pandas SQL.

In [11]:
df.to_csv('DFforSQL.csv', index = False) #Saving df to csv to turn into a database for SQLite

Now that this is saved, in the next notebook I will apply the DF to an SQL database and start calling queries to explore the data.